In [ ]:
#use a ratio ofthe cbg's income level to the highest income level, to find how many in that cbg amy use bus - as a percentage of the total number of ODs

In [9]:
import pandas as pd
from gtfs_functions import Feed
import numpy as np

gtfs_path = '/home/rishav/Programs/routing/data/CARTA_GTFS2024'
# Load data
income_df = pd.read_csv('/home/rishav/Programs/routing/data/income_tn_all.csv', index_col=0) 
income_df['estimate'] = income_df['estimate'].astype(float)

od_df = pd.read_csv('/home/rishav/Programs/routing/outputs/lodes_2021-01-05_transit_drive2.csv', index_col=0)

od_df = od_df[od_df['transit_time'] > od_df['drive_time']]
od_df = od_df[od_df['walk_time'] > od_df['drive_time']]
od_df = od_df[od_df['walk_time'] >= od_df['transit_time']]

# Merge OD data with income data on CBG
data = pd.merge(od_df, income_df, left_on='h_geocode', right_on='GEOID', how='left')
data = data.dropna(subset='estimate')

In [10]:
max_income = income_df['estimate'].max()
data['income_ratio'] = data['estimate'] / max_income

In [11]:
# Normalize travel times
data['normalized_drive_time'] = data['drive_time'] / data['drive_time'].max()
data['normalized_transit_time'] = data['transit_time'] / data['transit_time'].max()

# Assign high penalty to missing bus times
high_penalty = 10  # Arbitrary high value; adjust as needed
data['normalized_transit_time'].fillna(high_penalty, inplace=True)


In [12]:
# Weights (tune these based on your specific needs or model calibration)
income_weight = 0.5
transit_time_weight = -0.5  # Negative because shorter times should increase bus usage
drive_time_weight = 0.5

# Calculate estimated bus usage probability
data['bus_usage_probability'] = (
    income_weight * data['income_ratio'] +
    transit_time_weight * data['normalized_transit_time'] +
    drive_time_weight * data['normalized_drive_time']
)

# Normalize probability to [0, 1] range
data['bus_usage_probability'] = (data['bus_usage_probability'] - data['bus_usage_probability'].min()) / (
        data['bus_usage_probability'].max() - data['bus_usage_probability'].min())


In [13]:
od_count_per_cbg = od_df.groupby('h_geocode').size().reset_index(name='total_od_count')
data = pd.merge(data, od_count_per_cbg, on='h_geocode', how='left')
data['estimated_bus_users'] = data['bus_usage_probability'] * data['total_od_count']


In [14]:
trips_df = pd.read_csv(f'{gtfs_path}/trips.txt')
stop_times_df = pd.read_csv(f'{gtfs_path}/stop_times.txt')
stop_times_df = stop_times_df[stop_times_df['departure_time']<'24:00:00']


# Convert arrival and departure times to a proper time format if necessary
stop_times_df['arrival_time'] = pd.to_datetime(stop_times_df['arrival_time'])
stop_times_df['departure_time'] = pd.to_datetime(stop_times_df['departure_time'])

# Assume you are interested in a specific time window
start_time = pd.to_datetime('06:00:00')
end_time = pd.to_datetime('10:00:00')

# Filter stop_times for trips that are operating within the time window
relevant_trips = stop_times_df[(stop_times_df['arrival_time'] >= start_time) & (stop_times_df['departure_time'] <= end_time)]

# Count the number of unique trips operating during this time
num_buses = relevant_trips['trip_id'].nunique()


In [15]:
# Merge trips data with the relevant trips to get route information
route_buses = relevant_trips.merge(trips_df, on='trip_id', how='left')

# Count the number of unique trips (buses) for each route
num_buses_per_route = route_buses.groupby('route_id')['trip_id'].nunique().reset_index(name='num_buses')


In [16]:
# Merge the number of buses per route back into your main dataset
data = pd.merge(data, num_buses_per_route, left_on='route1', right_on='route_id', how='left')

# Fill NaN values for routes without bus service in the time window with 0
data['num_buses'].fillna(0, inplace=True)


In [18]:
# Assign disability status
data['is_disabled'] = np.random.rand(len(data)) < 0.005  # 0.5% probability of being disabled

data['bus_capacity'] = 50
data['max_bus_capacity'] = data['bus_capacity'] * data['num_buses']
data['adjusted_bus_users'] = np.floor(data[['estimated_bus_users', 'max_bus_capacity']].min(axis=1))


In [19]:
data['random_number'] = np.random.rand(len(data))  # Generates a random number between 0 and 1 for each row
data['choice'] = np.where(data['random_number'] < data['bus_usage_probability'], 'Bus', 'Car')
data['micro_transit_eligible'] = np.where(data['is_disabled'], np.random.rand(len(data)) < 0.3, False)  # 30% of disabled population
data['choice'] = np.where(data['micro_transit_eligible'], 'Micro-Transit', data['choice'])


In [20]:
# Keep the existing code for adjusting choices based on bus capacity
bus_user_counts = data.groupby(["route_id", "choice"]).size().unstack(fill_value=0)

for route_id, row in bus_user_counts.iterrows():
    if row.get("Bus", 0) > data.loc[data["route_id"] == route_id, "max_bus_capacity"].values[0]:
        # Too many bus users, need to adjust
        excess = row["Bus"] - data.loc[data["route_id"] == route_id, "max_bus_capacity"].values[0]
        bus_users = data[(data["route_id"] == route_id) & (data["choice"] == "Bus")]
        drop_indices = bus_users.sample(n=int(excess)).index  # Randomly select excess users to switch to 'Car' or 'Micro-Transit'
        alternative_choice = np.where(data.loc[drop_indices, 'micro_transit_eligible'], 'Micro-Transit', 'Car')
        data.loc[drop_indices, "choice"] = alternative_choice


In [23]:
data[data['choice'] == 'Micro-Transit'].to_csv('outputs/micro-transit_users.csv')